# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [125]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [126]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)
np.linalg.matrix_rank(a)

6

In [127]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [130]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L @ u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [42]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [43]:
np.linalg.matrix_rank(a1)

6

In [62]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/Users/fedorzavialov/PycharmProjects/СompPhys/venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/Users/fedorzavialov/PycharmProjects/СompPhys/venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/Users/fedorzavialov/PycharmProjects/СompPhys/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/Users/fedorzavialov/PycharmProjects/СompPhys/venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [63]:
from numpy.linalg import det
N = a.shape[1]
minors_a = np.zeros(N); minors_a1 = minors_a.copy()
for j in range(N):
    minors_a[j] = det(a[:j+1, :j+1]); minors_a1[j] = det(a1[:j+1, :j+1])
print(minors_a); print(minors_a1)

[ 3.000e+00 -3.375e+00 -8.860e-01  1.947e-02  1.573e-05 -2.492e-10]
[ 3.000e+00  0.000e+00 -8.033e+00 -4.935e-01  9.030e-04  2.318e-08]


С главными минорами матрицы $a$ все хорошо, они все отличны от нуля, можно применять $LU$-разложение. С матрицей $a_1$ проблемы: второй главный минор матрицы оказался равным 0. Поэтому стандартное $LU$-разложение не работает, нужно использовать $LUP$-разложение.

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [267]:
def diy_lup(a):
    from numpy.linalg import inv
    N = a.shape[0]
    
    u = a.copy()
    PL = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        pivot = np.argmax(np.abs(u[j:, j]))
        p = np.zeros((N, N))
        for m in range(N):
            for n in range(N):
                if m != pivot+j and m != j:
                    if m == n:
                        p[m, n] = 1
                p[j, pivot+j] = 1
                p[pivot+j, j] = 1
        u = p @ u
        P = p @ P
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        
        PL = PL @ inv(p) @ lam
    L = P @ PL
    return L, u, P

Проверим эту модицифированную функцию, дающую $LUP$-разложение, на матрице $a_1$.

In [268]:
from numpy.linalg import inv
L, u, P = diy_lup(a1)

print(L, "\n"); print(u, "\n");

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0.088 0.514 1.    0.   ]
 [1.    0.941 0.038 0.208 0.641 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.776e-17 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  2.220e-16 -1.427e-17  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00 -1.424e-16  3.391e-18  0.000e+00  0.000e+00 -1.233e-05]] 



In [269]:
def re_lup(L, u, P):
    return inv(P) @ L @ u

Проверим, действительно ли матрицы $a$ и $a_1$ хорошо раскладываются $LUP$-разложением.

In [270]:
from numpy.testing import assert_allclose

L1, u1, P1 = diy_lup(a1)
assert_allclose(re_lup(L1, u1, P1), a1)

In [271]:
from numpy.testing import assert_allclose

L, u, P = diy_lup(a)
assert_allclose(re_lup(L, u, P), a)